<a href="https://colab.research.google.com/github/AmanShukla111/First_repo/blob/main/EmotionIntensityRegulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers pandas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00


In [67]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset

emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

paws_dataset = load_dataset('paws', 'labeled_final')
paws_df = pd.DataFrame({'input_text': paws_dataset['train']['sentence1'], 'target_text': paws_dataset['train']['sentence2'], 'label': paws_dataset['train']['label']})

In [68]:
paws_df = paws_df[paws_df['label'] == 1]
paws_df = paws_df.drop(columns=['label'])
paws_df.shape

(21829, 2)

In [69]:
def get_emotions(text):
    result = emotion_classifier(text)
    dominant_emotion = max(result, key=lambda x: x['score'])
    return dominant_emotion['label']

In [70]:
paws_df.head()

,input_text,target_text
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
5,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ..."
7,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...


In [60]:
for text in paws_df['input_text'].head():
    result = emotion_classifier(text)
    print(result)


[{'label': 'neutral', 'score': 0.594438910484314}]
[{'label': 'neutral', 'score': 0.7176297307014465}]
[{'label': 'optimism', 'score': 0.5017265677452087}]
[{'label': 'neutral', 'score': 0.5758771300315857}]
[{'label': 'neutral', 'score': 0.7429802417755127}]


In [71]:
paws_df['input_emotion'] = paws_df['input_text'].apply(get_emotions)
paws_df['target_emotion'] = paws_df['target_text'].apply(get_emotions)

In [72]:
paws_df.head()

,input_text,target_text,input_emotion,target_emotion
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,neutral,neutral
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,neutral,neutral
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,neutral,neutral
5,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",neutral,neutral
7,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,neutral,neutral


In [73]:
paws_df = paws_df[paws_df['input_emotion'] != paws_df['target_emotion']]
paws_df = paws_df[
    (paws_df['input_emotion'] != 'neutral') & (paws_df['target_emotion'] != 'neutral')]

In [74]:
decreasing_intensity_pairs = paws_df[paws_df.apply(
    lambda row: emotion_classifier(row['input_text'])[0]['score'] > emotion_classifier(row['target_text'])[0]['score'],
    axis=1)]

increasing_intensity_pairs = paws_df.drop(decreasing_intensity_pairs.index)
increasing_intensity_pairs[['input_text', 'target_text']] = increasing_intensity_pairs[
    ['target_text', 'input_text']].reset_index(drop=True)

decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(
    lambda row: f"{row['input_emotion']} to {row['target_emotion']}: {row['input_text']}", axis=1)

<ipython-input-74-900249e48b5c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(


In [75]:
final_paws_dataset = pd.concat([decreasing_intensity_pairs, increasing_intensity_pairs], ignore_index=True)

final_paws_dataset.to_csv('output_paws_dataset.csv', index=False)

In [76]:
paws_df1 = pd.read_csv('output_paws_dataset.csv')
paws_df1.head()

,input_text,target_text,input_emotion,target_emotion
0,approval to admiration: He was an atheist in e...,"He was an atheist in early life , but transfor...",approval,admiration
1,disappointment to annoyance: The Izvoarele Riv...,The river Izvoarele is a tributary of the Rive...,disappointment,annoyance
2,admiration to approval: Grégoire was born in V...,Grégoire was born in Vého near Lunéville as th...,admiration,approval
3,fear to disappointment: Thomas Thomas Kane 's ...,Thomas Kane 's first memorable encounter with ...,fear,disappointment
4,"optimism to approval: During the second year ,...","During the second year , students will have mo...",optimism,approval
